In [1]:
from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten, Lambda
from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.initializers import glorot_uniform as xav

from keras import backend as K
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

batch_size = 128
input_shape = ( x_train.shape[1:])
kernel_size = 3
filters = 1
latent_dim = 2
epochs = 6
use_mse = True
load_weights = False
train_test_split=0.8
alpha=0.1

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
print(x_train.shape)

def break_image(inputs):
  res = []
  for inp in inputs:
    entry = []
    for i in range(2, inp.shape[0]-2, 5):
      for j in range(2, inp.shape[1]-2, 5):
        m1 = np.zeros((5,5))
        m2 = np.zeros((5,5))
        m3 = np.zeros((5,5))
        m1[2][2] = inp[i][j]

        m2[1][1] = inp[i-1][j-1]
        m2[1][2] = inp[i-1][j]
        m2[1][3] = inp[i-1][j+1]
        m2[2][1] = inp[i][j-1]
        m2[2][3] = inp[i][j+1]
        m2[3][1] = inp[i+1][j-1]
        m2[3][2] = inp[i+1][j]
        m2[3][3] = inp[i+1][j+1]

        m3[0][0] = inp[i-2][j-2]
        m3[0][1] = inp[i-2][j-1]
        m3[0][2] = inp[i-2][j]
        m3[0][3] = inp[i-2][j+1]
        m3[0][4] = inp[i-2][j+2]
        m3[1][0] = inp[i-1][j-2]
        m3[1][4] = inp[i-1][j+2]
        m3[2][0] = inp[i][j-2]
        m3[2][4] = inp[i][j+2]
        m3[3][0] = inp[i+1][j-2]
        m3[3][4] = inp[i+1][j+2]
        m3[4][0] = inp[i+2][j-2]
        m3[4][0] = inp[i+2][j-1]
        m3[4][0] = inp[i+2][j]
        m3[4][0] = inp[i+2][j+1]
        m3[4][0] = inp[i+2][j+2]

        entry.append(np.array([m1,m2,m3]))
    entry = np.reshape(entry, newshape=(3,25,5,5))
    entry = np.reshape(entry, newshape=(3,25,25))
    res.append(entry)
  return np.reshape(res, (3,inputs.shape[0],25,25))

# def create_image(inputs):
#   res = []
#   for inp in inputs:
#     inp = np.reshape(inp, (25,3,5,5))
#     for temp in inp:
#       concat_entry.append(np.add(temp[0], np.add(temp[1], temp[2])))
#     entry = np.reshape(np.concatenate(concat_entry), (25,25))
#     res.append(entry)
#   return np.array(res)

test1 = break_image(x_test)
print(test1.shape)
# test2 = create_image(test1)
# print(test2.shape)

# plt.imshow(test2[0])

11493376/11490434 [==============================] - 1s 0us/step
(60000, 28, 28)
(3, 10000, 25, 25)


In [0]:
x_train = break_image(x_train)
x_test = break_image(x_test)
input_shape = (x_train.shape[2:])

In [0]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(z_log_var) * epsilon

In [5]:
print(input_shape)
xi = Input(shape=input_shape)
xj = Input(shape=input_shape)
xk = Input(shape=input_shape)

encoder_inputs = [xi, xj, xk]
'''for i in range(2):
    filters *= 2
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               activation='relu',
               strides=2,
               padding='same')(x)'''

# shape info needed to build decoder model
shape = K.int_shape(xi)


# generate latent vector Q(z|X)
xi = Flatten()(xi)
xj = Flatten()(xj)
xk = Flatten()(xk)

fc1 = Dense(512, activation='relu', kernel_initializer = xav(1))
xi = fc1(xi)
xj = fc1(xj)
xk = fc1(xk)

z_mean = Dense(128, name='z_mean', kernel_initializer = xav(1))
z_log_var = Dense(128, name='z_log_var', kernel_initializer = xav(1))

xi_mean = z_mean(xi)
xi_log_var = z_log_var(xi)
xj_mean = z_mean(xj)
xj_log_var = z_log_var(xj)
xk_mean = z_mean(xk)
xk_log_var = z_log_var(xk)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
zi = Lambda(sampling, output_shape=(latent_dim,))([xi_mean, xi_log_var])
zj = Lambda(sampling, output_shape=(latent_dim,))([xj_mean, xj_log_var])
zk = Lambda(sampling, output_shape=(latent_dim,))([xk_mean, xk_log_var])


encoder_output = [zi, xi_mean, xi_log_var, zj, xj_mean, xj_log_var, zk, xk_mean, xk_log_var]

# instantiate encoder model
encoder = Model(encoder_inputs, encoder_output, name='encoder')
encoder.summary()

(25, 25)
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 25, 25)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 25, 25)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 25, 25)       0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 625)          0           input_1[0][0]                    
___________________________________________________________________________________

In [6]:
zi_input = Input(shape=[128])
zj_input = Input(shape=[128])
zk_input = Input(shape=[128])

# zi_input = tf.expand_dims(zi_input, 0)
# zj_input = tf.expand_dims(zj_input, 0)
# zk_input = tf.expand_dims(zk_input, 0)

latent_input=[zi_input,zj_input,zk_input]

fc1 = Dense(512, activation = 'relu', kernel_initializer = xav(1))
fc2 = Dense(shape[1] * shape[2], activation = 'sigmoid', kernel_initializer = xav(1))
reshape = Reshape((shape[1], shape[2]))

zi_outputs = reshape(fc2(fc1(zi_input)))
zj_outputs = reshape(fc2(fc1(zj_input)))
zk_outputs = reshape(fc2(fc1(zk_input)))

# use Conv2DTranspose to reverse the conv layers from the encoder

'''
l1 = Conv2DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        activation='relu',
                        strides=2,
                        padding='same')

l2 = Conv2DTranspose(filters=1,
                              kernel_size=kernel_size,
                              activation='sigmoid',
                              padding='same')

xi = fc2(zi_input)
xj = fc2(zj_input)
xk = fc2(zk_input)


for i in range(2):
    xi = l1(xi)
    filters //= 2

zi_outputs = l2(xi)

for i in range(2):
    xj = l1(xj)
    filters //= 2

zj_outputs = l2(xj)
    
for i in range(2):
    xk = l1(xk)
    filters //= 2

zk_outputs = l2(xk)

'''

outputs=[zi_outputs,zj_outputs,zk_outputs]


# instantiate decoder model
decoder = Model(latent_input,outputs, name='decoder')
decoder.summary()

Model: "decoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 512)          66048       input_4[0][0]                    
                                                                 input_5[0][0]              

In [0]:
inputs = encoder_inputs
outputs = decoder([encoder(inputs)[0], encoder(inputs)[3], encoder(inputs)[6]])
vae = Model(inputs, outputs, name='vae')
outputs = tf.concat(values = outputs, axis = 0)

In [0]:
from keras import backend

def func1(x, g_x, size = input_shape[1]):
    funq = K.zeros(x.shape)
    for i in range(size):
        funq = K.add(funq, K.square(K.multiply(x, (x-g_x))))
    
def l2(x, g_x, size = input_shape[1]):
    sum = K.zeros(x.shape) 
    for i in range(size):
        sum = K.add(sum, funq1(x, g_x, size))
    return K.min(sum/K.square(size))

def W2(mean1, var1, mean2, var2):
    return K.square((mean1-mean2)) + K.square((K.sqrt(var1)-K.sqrt(var2)))

def ranking_loss(encoder_outputs):
    _, m1, v1, __, m2, v2, _, m3, v3 = encoder_outputs
    return K.square(W2(m1,v1,m2,v2)) + K.exp(-1*W2(m1,v1,m3,v3))

def vae_loss(x = K.flatten(inputs), x_decoded_mean = K.flatten(outputs)):
    xent_loss = input_shape * metrics.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)
    return vae_loss

def kullback_leibler_divergence(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)
    if y_true > y_pred:
      return K.sum(y_true * K.log(y_true / y_pred), axis=-1)
    return K.sum(y_true * K.log(y_pred / y_true), axis=-1)

def vae_loss(y_true = encoder_inputs, y_pred = outputs):
  w_loss = K.mean(y_true * y_pred)
  return ranking_loss(encoder_output) + alpha*w_loss

#vae.add_loss(losses = kullback_leibler_divergence)
# def loss()

In [9]:
from keras import optimizers
from keras import metrics
import tensorflow as tf
from tensorflow.keras.losses import KLDivergence
adam=optimizers.Adam(lr = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
rmsprop=optimizers.RMSprop(lr=.01, rho=0.9)
vae.compile(optimizer=adam, loss=vae_loss, metrics=[metrics.AUC()])
vae.summary()

Model: "vae"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 25, 25)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 25, 25)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 25, 25)       0                                            
__________________________________________________________________________________________________
encoder (Model)                 [(None, 2), (None, 1 451840      input_1[0][0]                    
                                                                 input_2[0][0]                  

In [15]:
if load_weights:
    vae = vae.load_weights(args.weights)
else:
    # train the autoencoder
    loss = vae.fit(list(x_train),list(x_train),epochs=epochs,batch_size=batch_size,validation_data=(list(x_test),list(x_test)))
    vae.save_weights('vae_cnn_mnist.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/6
60000/60000 [==============================] - 14s 237us/step - loss: 2.7087 - decoder_loss: 0.9029 - decoder_auc_1: 0.2803 - decoder_auc_1_1: 0.2801 - decoder_auc_1_2: 0.2801 - val_loss: 2.7422 - val_decoder_loss: 0.9141 - val_decoder_auc_1: 0.3062 - val_decoder_auc_1_1: 0.3065 - val_decoder_auc_1_2: 0.3046
Epoch 2/6
60000/60000 [==============================] - 12s 201us/step - loss: 2.6852 - decoder_loss: 0.8951 - decoder_auc_1: 0.3214 - decoder_auc_1_1: 0.3217 - decoder_auc_1_2: 0.3215 - val_loss: 2.7617 - val_decoder_loss: 0.9206 - val_decoder_auc_1: 0.3300 - val_decoder_auc_1_1: 0.3304 - val_decoder_auc_1_2: 0.3297
Epoch 3/6
60000/60000 [==============================] - 12s 200us/step - loss: 2.6735 - decoder_loss: 0.8912 - decoder_auc_1: 0.3362 - decoder_auc_1_1: 0.3362 - decoder_auc_1_2: 0.3362 - val_loss: 2.7320 - val_decoder_loss: 0.9108 - val_decoder_auc_1: 0.3410 - val_decoder_auc_1_1: 0.3408 - val_decoder_auc_1_